In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# General Information about Data Sets

In [2]:
train2= pd.read_csv('cleaned_train.csv')
weather= pd.read_csv('cleaned_weather.csv')
spray2= pd.read_csv('cleaned_spray.csv')
train= train2.drop(['Year','Month','Day'], axis=1)
spray= spray2.drop(['Year','YearMonth','Day','YearWeek','Month'], axis=1)
train['Date']=pd.to_datetime(train['Date'])
spray['Date']=pd.to_datetime(spray['Date'])
weather['Date']=pd.to_datetime(weather['Date'])
weather['YearWeek']= (weather['Year'].astype(str)+weather['Week'].astype(str)).astype('int64')
weather['YearMonth']=(weather['Year'].astype(str)+weather['Month'].astype(str)).astype('int64')
weather.drop(['Week','Day'], axis=1, inplace=True)

print(train.info(),weather.info(),spray.info())

train.head()

spray.head()

weather.head()

# Weather & Train Data Sets - EDA and Feature Engineering

According to the researches, if the weather gets too hot and too dry, mosquitoes will not be as active and feeding as they usually are. But once the humidity increases they’re more hungry and biting more.Therefore humidity plays a key role in WVN Presence. We will add the relative humidity as a feature.

In [ ]:
# calculation of RELATIVE HUMIDITY
def rel_humidity(df,T, Td,Tw):
    
# Convert the air temperature and dew-point temperature to Celsius.(C=5*(F-32)/9)
    Tc= (5.0*(df[T]-32))/9.0
    Tdc= (5.0*(df[Td]-32))/9.0

#Calculate the saturated vapor pressure with a formula.  
    es=6.11*10.0**(7.5*Tc/(237.7+Tc))

#Find the actual vapor pressure with the same formula.
    e=6.11*10.0**(7.5*Tdc/(237.7+Tdc))
    
#Calculate the relative humidity.    
    df['RelHumidity']= round((e/es)*100)
    return df

rel_humidity(weather,'Tavg','DewPoint', 'WetBulb')

I will calculate time lags by giving higher weights to the most recent observed values.

def ema(df,col, span):
    df[f'{col}_{span}']= round(df[col].ewm(span=span, adjust=True).mean(),2)
    return df
ema_list= ['Tmax', 'DewPoint','WetBulb','PrecipTotal', 'StnPressure','SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed','RelHumidity']
span= [7,14]
for i in ema_list:
    ema(weather,i, 7)
    ema(weather,i, 14)

In [9]:
def ema(df,col, span):
    df[f'{col}_{span}']= round(df[col].ewm(span=span, adjust=True).mean(),2)
    return df
ema_list= ['Tmax', 'DewPoint','WetBulb','PrecipTotal', 'StnPressure','SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed','RelHumidity']
span= list(range(1,15))
for i in ema_list:
    for s in span:
        ema(weather,i,s)

weather.head()

In [11]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Columns: 160 entries, Date to RelHumidity_14
dtypes: datetime64[ns](1), float64(152), int64(7)
memory usage: 1.8 MB


In [12]:
w_train= train.merge(weather, on= ['Date'])
w_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9693 entries, 0 to 9692
Columns: 165 entries, Date to RelHumidity_14
dtypes: datetime64[ns](1), float64(154), int64(9), object(1)
memory usage: 12.3+ MB


**The below chart indicates CULEX RESTUANS species were most affected by temperature increases.**

Virus_Species_Month=w_train[w_train['WnvPresent']==1].groupby(['Tmax'])['Species'].value_counts().unstack().fillna(0)
Virus_Species_Month.plot.bar(figsize=(14,6))
plt.grid(False)
plt.legend(bbox_to_anchor=(1, 1), fontsize='small')
plt.xlabel('Max Temperature', fontsize=13)
plt.ylabel('Number of WN Virus', fontsize=13)
plt.title('WN Virus Distribution per Species and Maximum Temperature', fontsize=18, color='red')
plt.show() 

w_train=pd.get_dummies(w_train,drop_first=True)
w_train.head()

In [14]:
w_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9693 entries, 0 to 9692
Columns: 170 entries, Date to Species_CULEX TERRITANS
dtypes: datetime64[ns](1), float64(154), int64(9), uint8(6)
memory usage: 12.3 MB


In [15]:
w_train.describe()

,Latitude,Longitude,NumMosquitos,WnvPresent,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,...,RelHumidity_11,RelHumidity_12,RelHumidity_13,RelHumidity_14,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
count,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,...,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000,9693.000000
mean,41.847618,-87.702823,10.210564,0.051893,81.344888,63.036212,72.442381,2.528526,59.382389,64.216032,...,64.079322,64.047814,64.017459,63.988206,0.230991,0.461054,0.275663,0.008769,0.000619,0.022800
std,0.109416,0.093464,13.138722,0.221823,8.338857,7.551025,7.507838,6.546048,7.890180,7.419962,...,4.719834,4.574587,4.442262,4.320905,0.421489,0.498507,0.446871,0.093237,0.024873,0.149273
min,41.644612,-87.930995,1.000000,0.000000,57.000000,42.000000,51.000000,-12.000000,39.000000,33.500000,...,54.880000,54.610000,54.370000,54.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.750498,-87.760886,2.000000,0.000000,78.000000,59.000000,69.000000,-2.000000,54.000000,60.500000,...,61.240000,61.180000,60.910000,60.790000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,41.867108,-87.698457,4.000000,0.000000,83.000000,65.000000,74.000000,4.000000,59.500000,65.000000,...,63.860000,63.810000,63.850000,63.770000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,41.954690,-87.642984,13.000000,0.000000,87.000000,69.000000,78.000000,8.000000,66.500000,69.500000,...,66.070000,66.220000,66.370000,66.350000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,42.017430,-87.531635,50.000000,1.000000,96.000000,77.000000,85.000000,18.000000,73.000000,76.000000,...,75.910000,75.520000,75.170000,74.840000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
print('\n','The Percantage of Virus per Each Species  ','\n')
print('CULEX PIPIENS: ', round(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS'].sum()/len(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS'])*100,2))
print('CULEX RESTUANS: ', round(w_train[w_train.WnvPresent==1]['Species_CULEX RESTUANS'].sum()/len(w_train[w_train.WnvPresent==1]['Species_CULEX RESTUANS'])*100,2))
print('CULEX PIPIENS/RESTUANS: ', round(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS/RESTUANS'].sum()/len(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS/RESTUANS'])*100),2)


 The Percantage of Virus per Each Species   

CULEX PIPIENS:  41.35
CULEX RESTUANS:  9.74
CULEX PIPIENS/RESTUANS:  49.0 2


**The below Correlation report indicates Relative Humidity is highly correlated with WN Virus, consequently Dew Temperature and Wet Bulb. Also their 7 and 14 days Exponential Mean Averages have higher correlation than the actual observation values. This must be because of the incubation period.**

figure = plt.figure(figsize=(10,40))
sns.heatmap(w_train.corr()[['WnvPresent']].sort_values('WnvPresent',ascending=False),annot=True, cmap='YlGnBu')

**We will check if there is multicollinearity among the features and fix them.**

In [17]:
pd.options.mode.use_inf_as_na = True
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [19]:
X = w_train.drop('WnvPresent', axis=1)
y = w_train['WnvPresent']
X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.3)
final_iv, IV = data_vars(X_train, y_train)

C:\Users\laptop\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


**After dropping 35 useless features for prediction, 134 features left.** 

In [22]:
features = list(IV[(IV['IV'] >= 0.01) & (IV['IV'] <= 0.8)]['VAR_NAME'])
X2 = X_train[features]
X2.head()

,AvgSpeed,AvgSpeed_1,AvgSpeed_10,AvgSpeed_11,AvgSpeed_12,AvgSpeed_13,AvgSpeed_14,AvgSpeed_2,AvgSpeed_3,AvgSpeed_4,...,WetBulb_3,WetBulb_4,WetBulb_5,WetBulb_6,WetBulb_7,WetBulb_8,WetBulb_9,Year,YearMonth,YearWeek
9353,9.60,9.60,8.95,8.88,8.81,8.75,8.70,9.29,9.36,9.38,...,67.39,67.79,67.90,67.88,67.82,67.74,67.66,2013,20139,201337
5164,4.80,4.80,6.39,6.37,6.37,6.37,6.38,6.60,6.84,6.78,...,58.79,59.44,59.91,60.26,60.51,60.69,60.82,2009,20099,200938
3383,11.05,11.05,8.83,8.83,8.84,8.85,8.86,10.28,9.78,9.42,...,49.94,50.76,51.36,51.81,52.16,52.41,52.61,2009,20096,200923
3132,5.25,5.25,8.34,8.39,8.44,8.47,8.51,6.23,6.96,7.43,...,60.01,59.80,59.64,59.53,59.46,59.41,59.37,2007,200710,200740
3726,8.10,8.10,6.45,6.51,6.57,6.63,6.68,7.00,6.52,6.31,...,53.04,56.69,58.92,60.32,61.22,61.78,62.12,2009,20096,200926


In [23]:
X2.isnull().any().sum()

0

In [24]:
def iterate_vif(df, vif_threshold=5, max_vif=6):
    count = 0
    while max_vif > vif_threshold:
        count += 1
        print("Iteration # "+str(count))
        vif = pd.DataFrame()
        vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        vif["features"] = df.columns
        #print(vif["VIFactor"])
        #print(vif["features"])

        if vif['VIFactor'].max() > vif_threshold:
            print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], vif['VIFactor'].max()))
            df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
            max_vif = vif['VIFactor'].max()
        else:
            print('Complete')
            return df, vif.sort_values('VIFactor')

final_df, final_vif =  iterate_vif(X2)

Iteration # 1


C:\Users\laptop\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Removing StnPressure_12 with VIF of 9007199254740992.000000
Iteration # 2
Removing SeaLevel_12 with VIF of 9007199254740992.000000
Iteration # 3
Complete


**There were 2 features with high multicollinearity and we dropped them. In total we have 132 features.**

In [25]:
X_train=final_df
X_train.head()

,AvgSpeed,AvgSpeed_1,AvgSpeed_10,AvgSpeed_11,AvgSpeed_12,AvgSpeed_13,AvgSpeed_14,AvgSpeed_2,AvgSpeed_3,AvgSpeed_4,...,WetBulb_3,WetBulb_4,WetBulb_5,WetBulb_6,WetBulb_7,WetBulb_8,WetBulb_9,Year,YearMonth,YearWeek
9353,9.60,9.60,8.95,8.88,8.81,8.75,8.70,9.29,9.36,9.38,...,67.39,67.79,67.90,67.88,67.82,67.74,67.66,2013,20139,201337
5164,4.80,4.80,6.39,6.37,6.37,6.37,6.38,6.60,6.84,6.78,...,58.79,59.44,59.91,60.26,60.51,60.69,60.82,2009,20099,200938
3383,11.05,11.05,8.83,8.83,8.84,8.85,8.86,10.28,9.78,9.42,...,49.94,50.76,51.36,51.81,52.16,52.41,52.61,2009,20096,200923
3132,5.25,5.25,8.34,8.39,8.44,8.47,8.51,6.23,6.96,7.43,...,60.01,59.80,59.64,59.53,59.46,59.41,59.37,2007,200710,200740
3726,8.10,8.10,6.45,6.51,6.57,6.63,6.68,7.00,6.52,6.31,...,53.04,56.69,58.92,60.32,61.22,61.78,62.12,2009,20096,200926


In [30]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6785 entries, 9353 to 5618
Columns: 132 entries, AvgSpeed to YearWeek
dtypes: float64(125), int64(5), uint8(2)
memory usage: 6.8 MB


In [26]:
final_vif

,VIFactor,features
90,1.227781,Species_CULEX RESTUANS
89,1.233482,Species_CULEX PIPIENS
34,1.945675,Longitude
33,1.965970,Latitude
0,NaN,AvgSpeed
...,...,...
127,NaN,WetBulb_8
128,NaN,WetBulb_9
129,NaN,Year
130,NaN,YearMonth


In [27]:
final_vif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132 entries, 90 to 131
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   VIFactor  4 non-null      float64
 1   features  132 non-null    object 
dtypes: float64(1), object(1)
memory usage: 3.1+ KB


# Spray & Train Data

**Virus have been found in most of the trap locations. However spraying doesn't cover the entre trap and virus locations.**

def interpretation(iv):
        if iv < 0.02:
            return 'useless'
        elif iv < 0.1:
            return 'weak'
        elif iv < 0.3:
            return 'medium'
        elif iv < 0.5:
            return 'strong'
        else:
            return 'suspicious'
interpretation(IV.loc[ :'IV'])

In [ ]:
from sklearn.neighbors import KernelDensity
mapdata = np.loadtxt("mapdata_copyright_openstreetmap_contributors.txt")
traps = train[['Date','Longitude', 'Latitude', 'WnvPresent','NumMosquitos']]

alpha_cm = plt.cm.Reds
alpha_cm._init()
alpha_cm._lut[:-3,-1] = abs(np.logspace(0, 1, alpha_cm.N) / 10 - 1)[::-1]
aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
lon_lat_box = (-88, -87.5, 41.6, 42.1)

spray_loc= spray[['Longitude', 'Latitude']].values
kd = KernelDensity(bandwidth=0.010)
kd.fit(spray_loc)

xv,yv = np.meshgrid(np.linspace(-88, -87.5, 100), np.linspace(41.6, 42.1, 100))
gridpoints = np.array([xv.ravel(),yv.ravel()]).T
zv = np.exp(kd.score_samples(gridpoints).reshape(100,100))

plt.figure(figsize=(18,14))
plt.imshow(mapdata, cmap=plt.get_cmap('gray'), extent=lon_lat_box, aspect=aspect)
plt.imshow(zv,origin='lower',cmap=alpha_cm, extent=lon_lat_box, aspect=aspect)

virus_loc = train[train['WnvPresent']==1][['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(virus_loc[:,0], virus_loc[:,1], marker='o', color='yellow', label='Virus Locations')
trap_loc = train[['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(trap_loc[:,0], trap_loc[:,1], marker='.', color='green', label='Trap Locations')
plt.legend(fontsize=15)
plt.title('Locations of Sprayed Traps and Virus', fontsize=22, color='red')
plt.xlabel('Longitude', fontsize=15)
plt.ylabel('Latitude', fontsize=15)
plt.show()

In [ ]:
X = w_train.drop('WnvPresent', axis=1)
y = w_train['WnvPresent']
X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.3)
print(X_train.shape, y_train.shape)

In [ ]:
df_ivf, IV = data_vars(X_train,y_train)